In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
import hail as hl

In [3]:
hl.init(
    tmp_dir=(localfs_path+'tmp_hail'),
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '10G'}, # I don't know what should be here
    default_reference='GRCh38') 

2023-04-16 21:10:11.431 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0013:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230416-2109-0.2.109-b71b065e4bb6.log


## Version 1 - with NAs if the genotype was not included 

In [3]:
#read in polish alleles
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt/')

In [4]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-anno.mt')

In [5]:
mt.count()

(10733631, 788)

In [6]:
pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)
mt = mt.annotate_cols(phenotypes = pheno[mt.s])

2023-04-03 12:11:43.395 Hail: INFO: wrote table with 187 rows in 1 partition to /localfs/2150834/tmp_hail/persist_tableVU7J29UroC
2023-04-03 12:11:44.589 Hail: INFO: Reading table to impute column types
2023-04-03 12:11:45.531 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading field 'kinship' as type str (imputed)
  Loading field 'disease' as type str (imputed)
  Loading field 'phenotype' as type str (imputed)
  Loading field 'add_pheno' as type str (imputed)
  Loading field 'heavy_tics' as type str (imputed)
  Loading field 'heavy_tics_familial' as type str (imputed)
  Loading field 'GTS_ASD_group' as type str (imputed)
  Loading field 'nonCTD' as type str (imputed)


In [8]:
test = mt.filter_cols(mt.phenotypes.family == '.')

In [11]:
test.aggregate_cols(hl.agg.counter(test.phenotypes.sex))

{'F': 6, 'M': 34}

In [ ]:
test.aggregate_cols(hl.agg.counter(test.phenotypes.add_pheno))

In [5]:
poles = poles.rows()
poles = hl.MatrixTable.from_rows_table(poles)
poles = poles.annotate_cols(col_idx = hl.missing(hl.tint32))
poles = poles.key_cols_by(poles.col_idx)

In [31]:
polish_samples = []

for i in range(1,1001):
    polish_samples.append(str(i)+'_polish')

In [6]:
polish_columns = hl.utils.range_matrix_table(n_rows=0, n_cols=1000)
polish_columns = polish_columns.annotate_rows(locus = hl.locus(hl.missing(hl.tstr), hl.missing(hl.tint32), reference_genome='GRCh38'),
                                              alleles = hl.missing(hl.tarray(hl.tstr)))
polish_columns = polish_columns.key_rows_by(polish_columns.locus, polish_columns.alleles)

In [7]:
polish_columns.write(localfs_path+'poles-1.mt')

2023-03-31 12:24:13.173 Hail: INFO: wrote matrix table with 0 rows and 1000 columns in 0 partitions to /localfs/2150588/poles-1.mt


In [8]:
poles = poles.union_cols(polish_columns, row_join_type='outer')

In [9]:
poles.write(localfs_path+'poles-2.mt')

2023-03-31 12:26:13.000 Hail: INFO: wrote matrix table with 21613392 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-2.mt


In [12]:
poles = hl.read_matrix_table(localfs_path+'poles-2.mt')

In [18]:
poles = poles.filter_rows(hl.is_defined(mt.rows()[poles.row_key])) #change this! create the matrix table with rows from the MT and draw based on poles. If not defined - assign ZERO (ask Marcin)

In [19]:
poles = poles.checkpoint(localfs_path+'poles-3.mt')

2023-03-31 12:30:38.735 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-3.mt


In [20]:
poles = poles.annotate_entries(AD = hl.missing(hl.tarray(hl.tint32)),
                         DP = hl.missing(hl.tint32),
                         GQ = hl.missing(hl.tint32),
                         GT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         MIN_DP = hl.missing(hl.tint32),
                         PGT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         PID = hl.missing(hl.tstr),
                         PL = hl.missing(hl.tarray(hl.tint32)),
                         PS = hl.missing(hl.tint32),
                         RGQ = hl.missing(hl.tint32),
                         SB = hl.missing(hl.tarray(hl.tint32)))

In [21]:
poles = poles.checkpoint(localfs_path+'poles-4.mt')

2023-03-31 12:39:44.868 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-4.mt
    Total size: 3.58 GiB
    * Rows/entries: 3.58 GiB
    * Columns: 3.81 KiB
    * Globals: 11.00 B
    * Smallest partition: 1901375 rows (1.72 GiB)
    * Largest partition:  2026979 rows (1.86 GiB)


In [32]:
poles = poles.annotate_cols(
    s = hl.literal(polish_samples)[poles.col_idx]
)

In [34]:
poles = poles.checkpoint(localfs_path+'poles-6.mt')

2023-03-31 12:46:15.696 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-6.mt


In [35]:
poles = poles.key_cols_by(poles.s)

In [36]:
poles = poles.checkpoint(localfs_path+'poles-7.mt')

2023-03-31 12:48:18.846 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-7.mt


In [38]:
poles.count()

(3928354, 1000)

In [39]:
poles.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls.mt')

2023-03-31 13:04:01.418 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls.mt


In [3]:
#read in polish alleles
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt/')

In [40]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-split.mt')

In [41]:
mt.count()

(10733631, 788)

In [5]:
poles = poles.rows()
poles = hl.MatrixTable.from_rows_table(poles)
poles = poles.annotate_cols(col_idx = hl.missing(hl.tint32))
poles = poles.key_cols_by(poles.col_idx)

In [31]:
polish_samples = []

for i in range(1,1001):
    polish_samples.append(str(i)+'_polish')

In [6]:
polish_columns = hl.utils.range_matrix_table(n_rows=0, n_cols=1000)
polish_columns = polish_columns.annotate_rows(locus = hl.locus(hl.missing(hl.tstr), hl.missing(hl.tint32), reference_genome='GRCh38'),
                                              alleles = hl.missing(hl.tarray(hl.tstr)))
polish_columns = polish_columns.key_rows_by(polish_columns.locus, polish_columns.alleles)

In [7]:
polish_columns.write(localfs_path+'poles-1.mt')

2023-03-31 12:24:13.173 Hail: INFO: wrote matrix table with 0 rows and 1000 columns in 0 partitions to /localfs/2150588/poles-1.mt


In [8]:
poles = poles.union_cols(polish_columns, row_join_type='outer')

In [9]:
poles.write(localfs_path+'poles-2.mt')

2023-03-31 12:26:13.000 Hail: INFO: wrote matrix table with 21613392 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-2.mt


In [12]:
poles = hl.read_matrix_table(localfs_path+'poles-2.mt')

In [18]:
poles = poles.filter_rows(hl.is_defined(mt.rows()[poles.row_key])) #change this! create the matrix table with rows from the MT and draw based on poles. If not defined - assign ZERO (ask Marcin)

In [19]:
poles = poles.checkpoint(localfs_path+'poles-3.mt')

2023-03-31 12:30:38.735 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-3.mt


In [20]:
poles = poles.annotate_entries(AD = hl.missing(hl.tarray(hl.tint32)),
                         DP = hl.missing(hl.tint32),
                         GQ = hl.missing(hl.tint32),
                         GT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         MIN_DP = hl.missing(hl.tint32),
                         PGT = hl.call(hl.int32(hl.rand_bool(poles.info['AF'][0])), hl.int32(hl.rand_bool(poles.info['AF'][0]))),
                         PID = hl.missing(hl.tstr),
                         PL = hl.missing(hl.tarray(hl.tint32)),
                         PS = hl.missing(hl.tint32),
                         RGQ = hl.missing(hl.tint32),
                         SB = hl.missing(hl.tarray(hl.tint32)))

In [21]:
poles = poles.checkpoint(localfs_path+'poles-4.mt')

2023-03-31 12:39:44.868 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-4.mt
    Total size: 3.58 GiB
    * Rows/entries: 3.58 GiB
    * Columns: 3.81 KiB
    * Globals: 11.00 B
    * Smallest partition: 1901375 rows (1.72 GiB)
    * Largest partition:  2026979 rows (1.86 GiB)


In [32]:
poles = poles.annotate_cols(
    s = hl.literal(polish_samples)[poles.col_idx]
)

In [34]:
poles = poles.checkpoint(localfs_path+'poles-6.mt')

2023-03-31 12:46:15.696 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-6.mt


In [35]:
poles = poles.key_cols_by(poles.s)

In [36]:
poles = poles.checkpoint(localfs_path+'poles-7.mt')

2023-03-31 12:48:18.846 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2150588/poles-7.mt


In [38]:
poles.count()

(3928354, 1000)

In [39]:
poles.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls.mt')

2023-03-31 13:04:01.418 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls.mt


## Version 2 - with zeros

In [5]:
#read in polish alleles
polish = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/resources/polish-genomes/polish-genomes.mt/')

In [6]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-anno.mt')

In [7]:
mt = mt.rows()
mt = hl.MatrixTable.from_rows_table(mt)
mt = mt.annotate_cols(col_idx = hl.missing(hl.tint32))
mt = mt.key_cols_by(mt.col_idx)

In [8]:
polish_samples = []

for i in range(1,1001):
    polish_samples.append(str(i)+'_polish')

In [9]:
polish_columns = hl.utils.range_matrix_table(n_rows=0, n_cols=1000)
polish_columns = polish_columns.annotate_rows(locus = hl.locus(hl.missing(hl.tstr), hl.missing(hl.tint32), reference_genome='GRCh38'),
                                              alleles = hl.missing(hl.tarray(hl.tstr)))
polish_columns = polish_columns.key_rows_by(polish_columns.locus, polish_columns.alleles)

In [10]:
mt = mt.union_cols(polish_columns, row_join_type='outer')

In [ ]:
mt.write(localfs_path+'temp-1.mt')

In [ ]:
poles = hl.read_matrix_table(localfs_path+'temp-1.mt')

In [ ]:
poles.count()

In [ ]:
polish.count()

In [ ]:
gt = hl.if_else(
    hl.is_defined(polish.rows()[poles.row_key]),
    hl.call(
        hl.int32(hl.rand_bool(polish.rows()[poles.row_key].info.AF[0])), hl.int32(hl.rand_bool(polish.rows()[poles.row_key].info.AF[0]))
    ),
    hl.call(0, 0)
)

In [ ]:
poles = poles.annotate_entries(AD = hl.missing(hl.tarray(hl.tint32)),
                         DP = hl.missing(hl.tint32),
                         GQ = hl.missing(hl.tint32),
                         GT = gt,
                         MIN_DP = hl.missing(hl.tint32),
                         PGT = gt,
                         PID = hl.missing(hl.tstr),
                         PL = hl.missing(hl.tarray(hl.tint32)),
                         PS = hl.missing(hl.tint32),
                         RGQ = hl.missing(hl.tint32),
                         SB = hl.missing(hl.tarray(hl.tint32)))

In [ ]:
poles = poles.checkpoint(localfs_path+'poles.mt')

In [ ]:
poles = poles.annotate_cols(
    s = hl.literal(polish_samples)[poles.col_idx]
)

In [28]:
poles = poles.key_cols_by(poles.s)

In [29]:
poles.write('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls_zeros.mt')

2023-03-31 17:50:53.781 Hail: INFO: wrote matrix table with 10733531 rows and 1000 columns in 1995 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls_zeros.mt


### Make the simulated controls gender equal with the samples

In [4]:
poles_zeros = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls_zeros.mt')
poles = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/synthetic_controls.mt')

In [5]:
pl_mts = [poles_zeros,poles]

PAR coordinates in GrCh38: http://m.ensembl.org/info/genome/genebuild/human_PARS.html

In [6]:
par1 = [10001,2781479]
par2 = [155701383,156030895]

In [7]:
nonpars = [[1,10000],[2781480,155701382]]

In [8]:
polish_samples = []

for i in range(1,1001):
    polish_samples.append(str(i)+'_polish')

In [9]:
poles_females = polish_samples[0:150] 
poles_males = polish_samples[150:]

In [10]:
path_names = ['poles_zeros', 'poles']

In [11]:
for idx, mt in enumerate(pl_mts):
    mt = mt.annotate_cols(
        sex = hl.if_else(
            hl.array(poles_females).contains(mt.s),
            'F',
            'M'
        )
    )
    
    mt = mt.annotate_entries(
        GT = hl.if_else(
            mt.locus.contig == 'chrX',
            hl.if_else(
                mt['sex'] == 'F',
                mt.GT,
                hl.if_else(
                    mt.locus.position > nonpars[1][1],
                    mt.GT,
                    hl.if_else(
                        mt.locus.position < nonpars[0][1],
                        hl.if_else(
                            mt.locus.position > nonpars[0][0],
                            mt.GT,
                            hl.call(mt.GT[0], mt.GT[0])),
                        hl.call(mt.GT[0], mt.GT[0])))),
            mt.GT
        )
    )
    
    mt = mt.checkpoint(localfs_path+path_names[idx])  

2023-04-16 10:21:45.325 Hail: INFO: wrote matrix table with 10733531 rows and 1000 columns in 1995 partitions to /localfs/2307051/poles_zeros
2023-04-16 10:24:44.370 Hail: INFO: wrote matrix table with 3928354 rows and 1000 columns in 2 partitions to /localfs/2307051/poles


### Join with existing samples

In [12]:
mt = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-anno.mt'
)

In [13]:
mt = mt.drop(mt.group, mt.sample_qc)

In [14]:
mt = mt.select_entries(
    AD = mt.LAD,
    DP = mt.DP,
    GQ = mt.GQ,
    GT = mt.GT,
    MIN_DP = mt.MIN_DP,
    PGT = mt.LPGT,
    PID = mt.PID,
    PL = mt.LPL,
    PS = mt.PS,
    RGQ = mt.RGQ,
    SB = mt.SB
)

In [15]:
mt = mt.annotate_cols(sex = hl.missing(hl.tstr))

In [ ]:
for idx in [0,1]:
    pl = hl.read_matrix_table(localfs_path+path_names[idx])  
    pl = pl.drop(pl.col_idx)
    pl = mt.union_cols(pl)
    pl.write(localfs_path+path_names[idx]+'union.mt')